### 垃圾邮件分类

此部分主要是使用上个文件中手写的朴素贝叶斯分类器进行垃圾邮件（具体数据）分类，详细内容可查看源代码。

数据说明：

- 数据在当前目录下的email目录下
- ham目录下：表示正常邮件
- spam目录下：表示垃圾邮件
- 邮件内容为英文

In [1]:
from numpy import *

# 创建一个在所有文档中出现的不重复词的列表
# 使用了python的set数据类型，将词条列表输给set构造函数，set就会返回一个不重复的词表
def createVocabList(dataSet):
    vocabSet = set([])  # 创建一个空集
    for document in dataSet:
        vocabSet = vocabSet | set(document)  # 创建两个集合的并集
    return list(vocabSet)


# 词袋模型（文档中的词可以多次出现）
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1  # 每出现一次，对应的数目就加1
    return returnVec


# 朴素贝叶斯分类器训练函数
def trainNB0(trainMatrix, trainCategory):  # 文档矩阵、由每篇文档类别标签所构成的向量
    numTrainDocs = len(trainMatrix)  # 计算训练样本个数
    numWords = len(trainMatrix[0])  # 每个样本的词向量中的元素个数
    pAbusive = sum(trainCategory) / float(numTrainDocs)  # 计算p(1)

    p0Num = ones(numWords)  # 创建一个1填充的矩阵，并且元素个数与词向量的元素个数相同，用于记录正常词语
    p1Num = ones(numWords)  # 用于记录侮辱性词语

    p0Denom = 2.0
    p1Denom = 2.0

    for i in range(numTrainDocs):  # 遍历训练集中的所有文档
        if trainCategory[i] == 1:  # 如果类标签为1（即侮辱性文档）
            p1Num += trainMatrix[i]  # 每个侮辱性文档中每个词对应位置个数+1
            p1Denom += sum(trainMatrix[i])  # 每个侮辱性文档的总词数叠加
        else:  # 如果类标签为0，做同样操作
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num / p1Denom)  # 对每个元素除以该类别中的总词数
    p0Vect = log(p0Num / p0Denom)
    return p0Vect, p1Vect, pAbusive  # 返回两个向量一个概率


# 朴素贝叶斯分类函数
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):  # 要分类的向量以及使用函数trainNB0计算的三个概率
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)  # 两个向量相乘（对应元素相乘），然后将词汇表中所有词的对应值相加，然后将该值加到类别的对数概率上
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:  # 比较类别的概率
        return 1
    else:
        return 0

# 接受一个大字符串并将其解析为字符串列表
def textParse(bigString):
    import re
    listOfTokens = re.split(r'\W', bigString)  # \W* 表示任意非字符或数字的组合出现零或更多次
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]  # 字符串长度大于2，转为小写

# 对贝叶斯垃圾邮件分类器进行自动处理
def spamTest():
    # 定义一些空列表
    docList = []
    classList = []
    fullText = []
    for i in range(1, 26):
        wordList = textParse(open('email/spam/%d.txt' % i,errors='ignore').read()) # 导入文件夹spam下的文件，并解析为词列表
        docList.append(wordList)  # 将整个词列表加到docList列表中，关于append和extend的用法可自行查询下
        fullText.extend(wordList) # 将词列表中的元素加到fullText列表中
        classList.append(1)   # 垃圾邮件标记为1
        wordList = textParse(open('email/ham/%d.txt' % i,errors='ignore').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)

    vocabList = createVocabList(docList)  # 调用createVocabList函数创建一个在所有文档中出现的不重复词的列表
    trainingSet = list(range(50))
    testSet = []
    for i in range(10):
        randIndex = int(random.uniform(0, len(trainingSet))) # 0-50的随机整数
        testSet.append(trainingSet[randIndex])  # 在所有邮件中随机选取10个作为测试集
        del (trainingSet[randIndex]) # 删除用于作为测试集的邮件
    trainMat = []
    trainClasses = []
    for docIndex in trainingSet: # 遍历训练集
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))  # 用词袋模型对每个样本进行处理，返回邮件对应的词向量
        trainClasses.append(classList[docIndex])  # 训练样本的标签
    p0V, p1V, pSpam = trainNB0(array(trainMat), array(trainClasses))  # 调用trainNB0训练函数
    errorCount = 0  # 用于记录分类错误的个数
    for docIndex in testSet:  # 遍历测试集
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector), p0V, p1V, pSpam) != classList[docIndex]: # 分类结果与原标签不一致
            errorCount += 1 # 错误个数+1
            # print("分类错误：", docList[docIndex])
    # print('错误率为: ', float(errorCount) / len(testSet))
    return float(errorCount) / len(testSet)

# 测试（迭代10次求平均）
error = 0.0
for i in range(10):
    error += spamTest()
error = error / 10.0
print(error)

0.04
